In [1]:
# we scraper
from bs4 import BeautifulSoup

# request
import requests

# io
from io import BytesIO

# date parser and sleep
from dateutil.parser import parse
import time

# web driver
from datetime import datetime
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc

# reeading and exporting to excel files
import pandas as pd


# looping process monitoring
from tqdm import tqdm
import random

In [2]:
# class to determine ganjil genap
def is_date(string, fuzzy=False):
    try: 
        parse(string, fuzzy=fuzzy)
        return True
    except ValueError:
        return False

s = slice(11)

In [3]:
# reading all BL number from google sheet
from acquiringbl import takingBL
bl_list = takingBL("SEALAND")

In [4]:
len(bl_list)

36

In [5]:
dict_hasil = {}
current_dict = {}
list_dict = []
gagal = []

In [16]:
bl_list = ['234114361',
 '234211984',
 '234258035',
 '233967407',
 '234243536',
 '234258447',
 '234351198',
 '234353901',
 '234314302',
 '234451826',
 '234453071',
 '234487414',
 '234271117',
 '234633636',
 '234643474',
 '234363863',
 '234363791']

**FIX**


In [19]:
dict_hasil = {}
current_dict = {}
list_dict = []
gagal = []

# web scripting
options = Options()
options.add_argument("--window-size=1920,1280")
driver = uc.Chrome(use_subprocess=True)
driver.get('https://www.maersk.com/tracking/' + bl_list[random.randint(0, len(bl_list)-1)])

# wait to load all
time.sleep(2)

#click allow all
click_allowall = driver.find_element(By.XPATH,'//button[text()="Allow all"]')
click_allowall.click()

for i, bls in enumerate(tqdm(bl_list)):
    try:
        # access the web
        driver.get('https://www.maersk.com/tracking/' + bl_list[i])
        # sleep wait
        title = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='maersk-app']/div/div/div/dl")))
        time.sleep(3)
        
        # finding how many hide details in 1 page refering to how many container number in 1 bl
        soup1 = BeautifulSoup(driver.page_source, 'lxml')
        containers = soup1.find_all('div', {"data-test": "container"})
        number_of_ctr = len(containers)
        
        print(bls, "HAVE ", number_of_ctr, " CTR")

        if number_of_ctr > 1:
            titles = title.text.split('\n')
            titles.remove('View Shipment Details')
            print(titles)
            for a, item in enumerate(titles):
                #
                if a%2==0 :
                    case_judul = {titles[a]:titles[a+1]}
                    dict_hasil.update(case_judul)
            
            print("Hasil dari Append Judul")
            print(dict_hasil)
            if driver.find_element(By.CLASS_NAME, "track__feedback"):
                x=4
            else:
                x=3
            # clicking all view more
            for c in range(number_of_ctr):
                try:
                    print("clicking div no :",  x)
                    show_details2 = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/main/div/div/div/div/div[{}]/dl/button'.format(x))))
                    show_details_check = driver.find_element(By.XPATH, "/html/body/main/div/div/div/div/div[{}]/dl/button/span".format(x))
                    if show_details_check.text == 'Show details':
                        time.sleep(2)
                        show_details2.click()
                        print("clicked",x, show_details_check.text)
                        x=x+1
                                        
                    time.sleep(1)
                except Exception as e:
                    x=x+1

            time.sleep(2)
            # taking the whole page's html
            soup2 = BeautifulSoup(driver.page_source, 'lxml')
            
            table = soup2.find_all('div', {"data-test": "container"})
            milestones = []
            for tag in table:
                for ctri1, ctr_num1  in enumerate(tag.find_all('strong', {"data-test":'container-details-value'})):
                    temp_ctr_num1 = ctr_num1
                    milestones.append("Container Number")
                    milestones.append(temp_ctr_num1.text)
                for m, li_milestone in enumerate(tag.find_all('li', {"data-test":'transport-plan-item-complete'})):
                    span_history = li_milestone.find_all("span")
                    for c1, city1 in enumerate(li_milestone.find_all("strong")):
                        temp1 = city1.text
                    for milestone_span in span_history:
                        if is_date(milestone_span.text):
                            milestones.append(milestone_span.text[s])
                        else:
                            milestones.append(temp1 +" "+ milestone_span.text)
                            
                for n, li_expected in enumerate(tag.find_all('li', {"data-test":'transport-plan-item-future'})):
                    spans = li_expected.find_all("span")
                    for c2, city2 in enumerate(li_expected.find_all("strong")):
                        temp2 = city2.text
                    for expected_span in spans:
                        if is_date(expected_span.text):
                            milestones.append(expected_span.text[s])
                        else:
                            milestones.append(temp2 + " ESTIMATED " + expected_span.text)
                 
                for panjang in range(0, len(milestones), 2):
                    key = milestones[panjang]
                    value = milestones[panjang + 1]

                    if key == 'Container Number':
                        if current_dict:
                            list_dict.append(current_dict.copy())
                        current_dict = {'Container Number': value}
                    else:
                        current_dict[key] = value

                    current_dict.update(dict_hasil) 
                print(current_dict)
            list_dict.append(current_dict)
            current_dict = {}
            dict_hasil={}
            print(bls, ", completed!!")
        else:
            time.sleep(1)
            
            titles = title.text.split('\n')
            titles.remove('View Shipment Details')
            print(titles)
            for a, item in enumerate(titles):
                if a%2==0 :
                    case_judul = {titles[a]:titles[a+1]}
                    dict_hasil.update(case_judul)
            
            time.sleep(2)
            soup2 = BeautifulSoup(driver.page_source, 'lxml')
            
            table = soup2.find_all('div', {"data-test": "container"})
            milestones = []
            for tag in table:
                for ctri1, ctr_num1  in enumerate(tag.find_all('strong', {"data-test":'container-details-value'})):
                    temp_ctr_num1 = ctr_num1
                    milestones.append("Container Number")
                    milestones.append(temp_ctr_num1.text)

                for m, li_milestone in enumerate(tag.find_all('li', {"data-test":'transport-plan-item-complete'})):
                    span_history = li_milestone.find_all("span")
                    for c1, city1 in enumerate(li_milestone.find_all("strong")):
                        temp1 = city1.text
                    for milestone_span in span_history:
                        if is_date(milestone_span.text):
                            milestones.append(milestone_span.text[s])
                        else:
                            milestones.append(temp1 +" "+ milestone_span.text)
                            
                for n, li_expected in enumerate(tag.find_all('li', {"data-test":'transport-plan-item-future'})):
                    spans = li_expected.find_all("span")
                    for c2, city2 in enumerate(li_expected.find_all("strong")):
                        temp2 = city2.text
                    for expected_span in spans:
                        if is_date(expected_span.text):
                            milestones.append(expected_span.text[s])
                        else:
                            milestones.append(temp2 + " ESTIMATED " + expected_span.text)
            
            for t, milestone in enumerate(milestones):
                if milestone == "Container Number":
                    case_mlstn = {milestones[t]:milestones[t+1]}
                    current_dict.update(case_mlstn)
                elif t%2==0 :
                    case_mlstn = {milestones[t]:milestones[t+1]}
                    current_dict.update(case_mlstn)
            current_dict.update(dict_hasil) 
            print(current_dict)
            list_dict.append(current_dict)
            current_dict = {}
            dict_hasil={}
        
        unique_list = []
        for d in list_dict:
            if d not in unique_list:
                unique_list.append(d)
                
    except Exception as e:
        print(e)
        print("{} GAGAL!!".format(bls))
        gagal.append(bls)

  0%|          | 0/17 [00:00<?, ?it/s]

234114361 HAVE  3  CTR
['Bill of Lading number', '234114361', 'From', 'Jakarta', 'To', 'Xingang']
Hasil dari Append Judul
{'Bill of Lading number': '234114361', 'From': 'Jakarta', 'To': 'Xingang'}
clicking div no : 4
clicked 4 Hide details
clicking div no : 5
clicked 5 Hide details
clicking div no : 6


  6%|▌         | 1/17 [00:38<10:08, 38.01s/it]

{'Jakarta Gate outEmpty': '06 Dec 2023', 'Bill of Lading number': '234114361', 'From': 'Jakarta', 'To': 'Xingang', 'Jakarta Gate in': '08 Dec 2023', 'Jakarta Load': '11 Dec 2023', 'Jakarta Vessel departure': '11 Dec 2023', 'Inchon Discharge': '26 Dec 2023', 'Inchon Load': '26 Dec 2023', 'Inchon Vessel departure': '26 Dec 2023', 'Xingang Vessel arrival': '31 Dec 2023', 'Xingang Discharge': '31 Dec 2023', 'Xingang Gate out for delivery': '02 Jan 2024', 'Xingang Empty container return': '03 Jan 2024'}
{'Jakarta Gate outEmpty': '07 Dec 2023', 'Bill of Lading number': '234114361', 'From': 'Jakarta', 'To': 'Xingang', 'Jakarta Gate in': '09 Dec 2023', 'Jakarta Load': '11 Dec 2023', 'Jakarta Vessel departure': '11 Dec 2023', 'Inchon Discharge': '26 Dec 2023', 'Inchon Load': '26 Dec 2023', 'Inchon Vessel departure': '26 Dec 2023', 'Xingang Vessel arrival': '31 Dec 2023', 'Xingang Discharge': '31 Dec 2023', 'Xingang Gate out for delivery': '04 Jan 2024', 'Xingang Empty container return': '04 Jan

 12%|█▏        | 2/17 [00:46<05:08, 20.54s/it]

...

{'Jakarta Gate outEmpty': '23 Dec 2023', 'Jakarta Gate in': '25 Dec 2023', 'Jakarta Load': '29 Dec 2023', 'Jakarta Vessel departure': '29 Dec 2023', 'Tanjung Pelepas Vessel arrival': '31 Dec 2023', 'Tanjung Pelepas Discharge': '01 Jan 2024', 'Tanjung Pelepas Load': '05 Jan 2024', 'Tanjung Pelepas Vessel departure': '06 Jan 2024', 'Sandakan Discharge': '10 Jan 2024', 'Bill of Lading number': '234643474', 'From': 'Jakarta', 'To': 'Sandakan'}
234363863 HAVE  1  CTR
['Bill of Lading number', '234363863', 'From', 'Jakarta', 'To', 'Kuching']


 94%|█████████▍| 16/17 [04:46<00:14, 14.32s/it]

{'Jakarta Gate outEmpty': '14 Dec 2023', 'Jakarta Gate in': '15 Dec 2023', 'Jakarta Load': '19 Dec 2023', 'Jakarta Vessel departure': '19 Dec 2023', 'Tanjung Pelepas Vessel arrival': '20 Dec 2023', 'Tanjung Pelepas Discharge': '21 Dec 2023', 'Tanjung Pelepas Load': '26 Dec 2023', 'Tanjung Pelepas Vessel departure': '26 Dec 2023', 'Kuching Vessel arrival': '28 Dec 2023', 'Kuching Discharge': '30 Dec 2023', 'Kuching Gate out for delivery': '08 Jan 2024', 'Kuching Empty container return': '09 Jan 2024', 'Bill of Lading number': '234363863', 'From': 'Jakarta', 'To': 'Kuching'}
234363791 HAVE  1  CTR
['Bill of Lading number', '234363791', 'From', 'Jakarta', 'To', 'Bintulu']


100%|██████████| 17/17 [04:54<00:00, 17.35s/it]

{'Jakarta Gate outEmpty': '13 Dec 2023', 'Jakarta Gate in': '15 Dec 2023', 'Jakarta Load': '18 Dec 2023', 'Jakarta Vessel departure': '19 Dec 2023', 'Tanjung Pelepas Vessel arrival': '20 Dec 2023', 'Tanjung Pelepas Discharge': '20 Dec 2023', 'Tanjung Pelepas Load': '06 Jan 2024', 'Tanjung Pelepas Vessel departure': '06 Jan 2024', 'Bintulu ESTIMATED Vessel arrival': '10 Jan 2024', 'Bill of Lading number': '234363791', 'From': 'Jakarta', 'To': 'Bintulu'}


In [20]:
unique_list

[{'Jakarta Gate outEmpty': '07 Dec 2023',
  'Bill of Lading number': '234114361',
  'From': 'Jakarta',
  'To': 'Xingang',
  'Jakarta Gate in': '09 Dec 2023',
  'Jakarta Load': '11 Dec 2023',
  'Jakarta Vessel departure': '11 Dec 2023',
  'Inchon Discharge': '26 Dec 2023',
  'Inchon Load': '26 Dec 2023',
  'Inchon Vessel departure': '26 Dec 2023',
  'Xingang Vessel arrival': '31 Dec 2023',
  'Xingang Discharge': '31 Dec 2023',
  'Xingang Gate out for delivery': '05 Jan 2024',
  'Xingang Empty container return': '04 Jan 2024'},
 {'Jakarta Gate outEmpty': '09 Dec 2023',
  'Jakarta Gate in': '10 Dec 2023',
  'Jakarta Load': '13 Dec 2023',
  'Jakarta Vessel departure': '13 Dec 2023',
  'Tanjung Pelepas Vessel arrival': '15 Dec 2023',
  'Tanjung Pelepas Discharge': '15 Dec 2023',
  'Tanjung Pelepas Load': '26 Dec 2023',
  'Tanjung Pelepas Vessel departure': '26 Dec 2023',
  'Kota Kinabalu Vessel arrival': '06 Jan 2024',
  'Kota Kinabalu Discharge': '08 Jan 2024',
  'Bill of Lading number': '

In [21]:
filtered_dict_list =[]
for dict_milestone in unique_list:
    # print(dict_milestone)
    kw = ['Bill of Lading number', 'Container Number', 'From', 'To', dict_milestone['From'], dict_milestone['To']]
    dict_key = []
    for keys in dict_milestone:
        dict_key.append(keys)
    # print("BEFORE", dict_key)
    new_list = [] 
    
    for key in dict_key:
        if any(substring in key for substring in kw):
            new_list.append(key)
    # print("AFTER",new_list)
    dict_milestone = {key: dict_milestone[key] for key in new_list}
    filtered_dict_list.append(dict_milestone)

In [22]:
filtered_dict_list

[{'Jakarta Gate outEmpty': '07 Dec 2023',
  'Bill of Lading number': '234114361',
  'From': 'Jakarta',
  'To': 'Xingang',
  'Jakarta Gate in': '09 Dec 2023',
  'Jakarta Load': '11 Dec 2023',
  'Jakarta Vessel departure': '11 Dec 2023',
  'Xingang Vessel arrival': '31 Dec 2023',
  'Xingang Discharge': '31 Dec 2023',
  'Xingang Gate out for delivery': '05 Jan 2024',
  'Xingang Empty container return': '04 Jan 2024'},
 {'Jakarta Gate outEmpty': '09 Dec 2023',
  'Jakarta Gate in': '10 Dec 2023',
  'Jakarta Load': '13 Dec 2023',
  'Jakarta Vessel departure': '13 Dec 2023',
  'Kota Kinabalu Vessel arrival': '06 Jan 2024',
  'Kota Kinabalu Discharge': '08 Jan 2024',
  'Bill of Lading number': '234211984',
  'From': 'Jakarta',
  'To': 'Kota Kinabalu'},
 {'Jakarta Gate outEmpty': '11 Dec 2023',
  'Jakarta Gate in': '13 Dec 2023',
  'Jakarta Load': '19 Dec 2023',
  'Jakarta Vessel departure': '19 Dec 2023',
  'Bintulu ESTIMATED Vessel arrival': '10 Jan 2024',
  'Bill of Lading number': '23425803

In [23]:
list_of_dict_fix = []
for filter_dict in filtered_dict_list:
    replacement_mapping = {
        filter_dict["From"]: 'Origin',
        filter_dict["To"]: 'Destination'
    }

    updated_dict = {}

    for key, value in filter_dict.items():
        for old_key, new_key in replacement_mapping.items():
            key = key.replace(old_key, new_key)
        updated_dict[key] = value

    list_of_dict_fix.append(updated_dict)

In [24]:
list_of_dict_fix

[{'Origin Gate outEmpty': '07 Dec 2023',
  'Bill of Lading number': '234114361',
  'From': 'Jakarta',
  'To': 'Xingang',
  'Origin Gate in': '09 Dec 2023',
  'Origin Load': '11 Dec 2023',
  'Origin Vessel departure': '11 Dec 2023',
  'Destination Vessel arrival': '31 Dec 2023',
  'Destination Discharge': '31 Dec 2023',
  'Destination Gate out for delivery': '05 Jan 2024',
  'Destination Empty container return': '04 Jan 2024'},
 {'Origin Gate outEmpty': '09 Dec 2023',
  'Origin Gate in': '10 Dec 2023',
  'Origin Load': '13 Dec 2023',
  'Origin Vessel departure': '13 Dec 2023',
  'Destination Vessel arrival': '06 Jan 2024',
  'Destination Discharge': '08 Jan 2024',
  'Bill of Lading number': '234211984',
  'From': 'Jakarta',
  'To': 'Kota Kinabalu'},
 {'Origin Gate outEmpty': '11 Dec 2023',
  'Origin Gate in': '13 Dec 2023',
  'Origin Load': '19 Dec 2023',
  'Origin Vessel departure': '19 Dec 2023',
  'Destination ESTIMATED Vessel arrival': '10 Jan 2024',
  'Bill of Lading number': '2342

In [25]:
list_of_dict_fix2 = []

for filter_dict in list_of_dict_fix:
    replacement_mapping = {
        "Bill of Lading number": "BL Number",
        "Origin Vessel departure": 'ATD',
        "Destination Discharge": 'ATA',
        "Destination Gate out": 'Container Release',
        "Destination Empty container return": 'Container Return',
    }

    updated_dict = {}

    for key, value in filter_dict.items():
        for old_key, new_key in replacement_mapping.items():
            if old_key in key:
                key = new_key

        updated_dict[key] = value

        if is_date(value):
            input_date = datetime.strptime(value, '%d %b %Y')
            updated_dict[key] = input_date.strftime("%Y-%m-%d")

    updated_dict["Liners"] = "SEALAND"

    list_of_dict_fix2.append(updated_dict)

In [26]:
list_of_dict_fix2

[{'Origin Gate outEmpty': '2023-12-07',
  'BL Number': '234114361',
  'From': 'Jakarta',
  'To': 'Xingang',
  'Origin Gate in': '2023-12-09',
  'Origin Load': '2023-12-11',
  'ATD': '2023-12-11',
  'Destination Vessel arrival': '2023-12-31',
  'ATA': '2023-12-31',
  'Container Release': '2024-01-05',
  'Container Return': '2024-01-04',
  'Liners': 'SEALAND'},
 {'Origin Gate outEmpty': '2023-12-09',
  'Origin Gate in': '2023-12-10',
  'Origin Load': '2023-12-13',
  'ATD': '2023-12-13',
  'Destination Vessel arrival': '2024-01-06',
  'ATA': '2024-01-08',
  'BL Number': '234211984',
  'From': 'Jakarta',
  'To': 'Kota Kinabalu',
  'Liners': 'SEALAND'},
 {'Origin Gate outEmpty': '2023-12-11',
  'Origin Gate in': '2023-12-13',
  'Origin Load': '2023-12-19',
  'ATD': '2023-12-19',
  'Destination ESTIMATED Vessel arrival': '2024-01-10',
  'BL Number': '234258035',
  'From': 'Jakarta',
  'To': 'Bintulu',
  'Liners': 'SEALAND'},
 {'Origin Gate outEmpty': '2023-12-12',
  'BL Number': '233967407',

In [27]:
# connect to mongodb
from mongoinit import mongo_table_initiation, insert_many_mongo

mongo_table_initiation()
insert_many_mongo(list_of_dict_fix2)

Today's Collection Name ===>  all_tracking_Jan-10-2024
Today's Collection Has Been Made
Inserting Many Complete!!


In [28]:
gagal

[]

In [16]:
len(gagal)

24